In [5]:
import os
import requests
import datetime
import pandas as pd
from aave_data import AAVE_DATA
from utils import date_to_timestamp

In [2]:
def fetch_aave_rates(
        reserve_id: str, 
        start_time: str, 
        resolution_in_hrs: int) -> pd.DataFrame:
    start_time = date_to_timestamp(start_time)
    url = ("https://aave-api-v2.aave.com" + 
            "/data/rates-history?" + 
            f"reserveId={reserve_id}&" +
            f"from={start_time}&" + 
            f"resolutionInHours={resolution_in_hrs}")
    
    try:
        response = requests.get(url)
        if response.status_code != 200:
            raise Exception(response.status_code)
        
        response_data = response.json()
        return pd.DataFrame(response_data)
    
    except Exception as error:
        raise Exception(error)

In [10]:
def format_aave_df(aave_df: pd.DataFrame) -> pd.DataFrame:
    aave_df['date-time'] = aave_df['x'].apply(
        lambda x: datetime.datetime(
            x['year'], x['month'] + 1, x['date'], x['hours']
        )
    )
    aave_df.drop(columns=['x'], inplace=True)
    aave_df = aave_df[["date-time"] + [col for col in aave_df.columns if col != "date-time"]]
    return aave_df

In [ ]:
timeStamp = date_to_timestamp("01-01-2000")

for data in AAVE_DATA:
    file_path = f"./aave_data/{data['market']}/{data['name']}.csv"
    if os.path.exists(file_path):
        user_input = input(f"{file_path} already exists! Do you want to fetch again and replace the file? (y/n): ").strip().lower()
        if user_input == 'y':
            response_df = fetch_aave_rates(data['rId'], timeStamp, 1)
            response_df = format_aave_df(response_df)
            response_df.to_csv(file_path, index=True)
            print(f"{file_path} has been replaced and saved.")
        else:
            print(f"Skipping {file_path}. No changes made.")
    else:
        response_df = fetch_aave_rates(data['rId'], timeStamp, 1)
        response_df = format_aave_df(response_df)
        response_df.to_csv(file_path, index=True)
        print(f"{file_path} saved.")